# **Exploratory Data Analysis Notebook**

## Objectives

* Inspect and understand the dataset via Exploratory Data Analysis. Check Data Types, Missing Data, Variables and Correlations. Perform statistic analysis to gain insight into data.

## Inputs

* Android_Malware.csv

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 


---

# Change working directory

Change the working directory from its current folder to its parent folder
* Access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

Make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Import Libraries and Load Dataset

In this section, all necessary standard libaries are imported to allow using their functions. The dataset is loaded to be able to access necessary data.

Import Libraries with necessary Settings

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
%matplotlib inline
sns.set(style="whitegrid")

Load Dataset from Inputs

In [ ]:
df = pd.read_csv("inputs/datasets/raw/Android_Malware.csv")

---

# Create Pandas Profiling Report

In this section, a pandas profiling report is created via ydata_profiling. The report serves as a general overview for the whole dataset and is saved as a pdf for future reference.

Import ydata_profiling and create and save Pandas Profile Report

In [ ]:
!pip show ipywidgets

In [ ]:
# Import Library
from ydata_profiling import ProfileReport

# Create Profiling Report
profile = ProfileReport(df=df, explorative=True)
profile.to_notebook_iframe()

# Save Report in Outputs Folder (create if not existing)
os.makedirs("outputs/eda", exist_ok=True)
profile.to_file("outputs/eda/EDA_Report.html")

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)
